In [79]:
# pip3 install s3fs

from sagemaker import Session
import sagemaker
import boto3
import re
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import os

In [80]:
role = get_execution_role()

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment/output'

train_path = f"s3://{bucket_name}/{prefix_name}/train"
validation_path = f"s3://{bucket_name}/{prefix_name}/validation"
test_path = f"s3://{bucket_name}/{prefix_name}/test"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [81]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, 
                                          framework='xgboost', 
                                          version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [83]:
x_train_data_file = f's3://{bucket_name}/{prefix_name}/train/train_data.csv'
print(x_train_data_file)
x_train_input = sagemaker.inputs.TrainingInput(x_train_data_file, content_type='csv')

validation_data_file = f's3://{bucket_name}/{prefix_name}/validation/validation_data.csv'
print(validation_data_file)
validation_input = sagemaker.inputs.TrainingInput(validation_data_file, content_type='text/csv')

s3://eliezerraj-908671954593-dataset/payment/output/train/train_data.csv
s3://eliezerraj-908671954593-dataset/payment/output/validation/validation_data.csv


In [84]:
import pandas as pd

df = pd.read_csv(validation_data_file)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head(5)

,0,6,0.1,1,1.1,722.0,8,279.25,60,284.18,185,310.72,3508
0,0,58,1,0,1,280.0,3,170.00,27,202.85,78,244.47,21058
1,0,14,0,1,1,128.0,5,295.40,24,293.54,24,293.54,148
2,0,56,1,0,1,121.0,8,320.75,49,338.57,128,311.86,326
3,0,121,1,0,1,264.0,12,269.92,55,303.67,156,285.86,140
4,1,208,0,1,1,654.0,16,415.94,65,416.69,206,376.13,69


In [20]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}'.format(bucket_name,
                                                                    prefix_name),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': x_train_input,
         'validation': validation_data_file})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: xgboost-2024-04-02-20-16-05-309


2024-04-02 20:16:05 Starting - Starting the training job...
2024-04-02 20:16:19 Starting - Preparing the instances for training......
2024-04-02 20:17:15 Downloading - Downloading input data...
2024-04-02 20:18:01 Downloading - Downloading the training image......
2024-04-02 20:19:02 Training - Training image download completed. Training in progress.
2024-04-02 20:19:02 Uploading - Uploading generated training modelArguments: train
[2024-04-02:20:18:54:INFO] Running standalone xgboost training.
[2024-04-02:20:18:54:INFO] File size need to be processed in the node: 1.69mb. Available memory size in the node: 8484.72mb
[2024-04-02:20:18:54:INFO] Determined delimiter of CSV input is ','
[20:18:54] S3DistributionType set as FullyReplicated
[20:18:54] 26989x12 matrix with 323868 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-04-02:20:18:54:INFO] Determined delimiter of CSV input is ','
[20:18:54] S3DistributionType set as FullyReplicated
[20:18:54] 7

In [21]:
# Deploy Model
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-04-02-20-20-30-706
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-04-02-20-20-30-706
INFO:sagemaker:Creating endpoint with name xgboost-2024-04-02-20-20-30-706


------!

In [22]:
#Evaluation
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [23]:
x_test_data_file = f's3://{bucket_name}/{prefix_name}/test/x_test.csv'
print(x_test_data_file)

y_test_data_file = f's3://{bucket_name}/{prefix_name}/test/y_test.csv'
print(y_test_data_file)

df_test_data_x = pd.read_csv(x_test_data_file,header=None)
df_test_data_y = pd.read_csv(y_test_data_file,header=None)

s3://eliezerraj-908671954593-dataset/payment/output/test/x_test.csv
s3://eliezerraj-908671954593-dataset/payment/output/test/y_test.csv


In [41]:
df_test_data_x.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11
0,23,1,0,1,278.0,7,249.86,57,296.05,197,276.11,732
1,108,0,1,1,117.0,2,257.00,63,396.13,99,369.65,1439
2,58,0,1,1,111.0,21,399.62,56,337.23,111,328.10,149
3,106,0,1,1,272.0,3,264.67,8,219.88,8,219.88,998
4,94,0,1,1,134.0,5,461.20,35,346.74,41,324.27,1334
5,79,0,1,1,118.0,6,205.67,43,305.47,130,280.25,854
6,79,0,1,1,192.0,2,164.00,52,269.15,205,281.82,66162
7,58,0,1,1,430.0,6,452.83,6,452.83,6,452.83,2778
8,377,0,1,1,1314.0,13,485.00,56,332.77,212,325.02,82
9,100,1,0,1,232.0,7,290.43,41,266.66,107,258.57,156


In [42]:
df_test_data_y.head(20)

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,1
9,0


In [26]:
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [27]:
predictions = predict(df_test_data_x, xgb_predictor)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [28]:
np.round(predictions)

array([0., 0., 0., ..., 0., 0., 0.])

In [29]:
pd.crosstab(index = df_test_data_y[0], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,3552,9
1,36,259


In [56]:
payment_data = [{'distance': 100,
                 'card_model_CHIP': 0.,
                 'card_model_VIRTUAL': 1.,
                 'card_type_CREDIT': 1.,
                 'amount': 2000,
                 'tx_1d': 9,
                 'avg_1d': 1365.00,
                 'tx_7d': 17,
                 'avg_7d': 263.529412,
                 'tx_30d': 28,
                 'avg_30d': 238.714286,
                 'time_btw_cc_tx': 97582.0}]
df_predict = pd.DataFrame(payment_data)
df_predict

,distance,card_model_CHIP,card_model_VIRTUAL,card_type_CREDIT,amount,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_cc_tx
0,100,0.0,1.0,1.0,2000,9,1365.0,17,263.529412,28,238.714286,97582.0


In [65]:
y_pred = xgb_predictor.predict(df_predict).decode('utf-8')
print(y_pred)
print(np.round(float(y_pred)))

0.8690098524093628
1.0


In [67]:
import boto3
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
endpoint_name='xgboost-2024-04-02-20-20-30-706'

In [68]:
# Endpoint invocation
payload = b"100., 0., 1., 1., 400., 365.0, 17., 263.529412, 28., 238.714286, 97582.0"

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)

y_predict = response["Body"].read().decode()
print(y_predict)
print(np.round(float(y_predict)))

0.6227468848228455
1.0


In [70]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2024-04-02-20-20-30-706
INFO:sagemaker:Deleting endpoint with name: xgboost-2024-04-02-20-20-30-706


In [75]:
validation_data_file.head(3)

AttributeError: 'str' object has no attribute 'head'

In [76]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                         'min_child_weight': ContinuousParameter(1, 10),
                         'alpha': ContinuousParameter(0, 2),
                         'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

In [77]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [78]:
tuner.fit({'train': x_train_input, 'validation': vaildation_input})

INFO:sagemaker:Creating hyperparameter tuning job with name: xgboost-240402-2155


......................................................................................................!


In [85]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [86]:
tuner.best_training_job()

'xgboost-240402-2155-012-1711fe34'

In [87]:
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')


2024-04-02 22:02:13 Starting - Found matching resource for reuse
2024-04-02 22:02:13 Downloading - Downloading the training image
2024-04-02 22:02:13 Training - Training image download completed. Training in progress.
2024-04-02 22:02:13 Uploading - Uploading generated training model
2024-04-02 22:02:13 Completed - Resource reused by training job: xgboost-240402-2155-015-1ea15572

INFO:sagemaker:Creating model with name: xgboost-2024-04-02-22-05-44-822


INFO:sagemaker:Creating endpoint-config with name xgboost-240402-2155-012-1711fe34
INFO:sagemaker:Creating endpoint with name xgboost-240402-2155-012-1711fe34


------!

In [88]:
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [89]:
x_test_data_file = f's3://{bucket_name}/{prefix_name}/test/x_test.csv'
print(x_test_data_file)

y_test_data_file = f's3://{bucket_name}/{prefix_name}/test/y_test.csv'
print(y_test_data_file)

df_test_data_x = pd.read_csv(x_test_data_file,header=None)
df_test_data_y = pd.read_csv(y_test_data_file,header=None)

s3://eliezerraj-908671954593-dataset/payment/output/test/x_test.csv
s3://eliezerraj-908671954593-dataset/payment/output/test/y_test.csv


In [93]:
pd.crosstab(index=df_test_data_y[0], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,3552,9
1,36,259
